### Title: Exercice 11.2
### Author: Jerock Kalala
### Date: November 14 2022
### Professor: David Kinney
### Working with databases

##### 1. Import sqlite3 and create a cursor

In [51]:
#Import sqlite3 and create a cursor
import statsmodels.api as sm
from pandas.io.sql import read_sql
import sqlite3

with sqlite3.connect(":memory:") as con:
    c = con.cursor()

Create the sensors table

In [52]:
c.execute('''CREATE TABLE sensors(date text, city text, code text, sensor_id real, temperature real)''')

Write a query to confirm your table was created – print the results

In [53]:
#Write a query to confirm your table was created – print the results
for table in c.execute("SELECT name FROM sqlite_master WHERE type = 'table'"):
    print("Table", table[0])

Table sensors


Insert random data into the table you created and then select all the results from the table (Select *)

In [54]:
#Insert random data into the table you created and then select all the results from the table (Select *)
c.execute("INSERT INTO sensors VALUES ('2016-02-18', 'Kinshasa', 'Pink', '243', '36.11')")
c.execute("SELECT * FROM sensors")
print(c.fetchone())

('2016-02-18', 'Kinshasa', 'Pink', 243.0, 36.11)


Drop the table and confirm it has been dropped based on your previous query

In [55]:
#Drop the table and confirm it has been dropped based on your previous query
con.execute("DROP TABLE sensors")
print("# of tables", c.execute("SELECT COUNT(*) FROM sqlite_master WHERE type = 'table'").fetchone()[0])

# of tables 0


Close your connection

In [56]:
#Close your connection
c.close()

#### <font color='green'>  2. Create a list of tuples to form the pandas DataFrame </font>

In [71]:
#Create a list of tuples to form the pandas DataFrame
data_loader = sm.datasets.sunspots.load_pandas()
df = data_loader.data
rows = [tuple(x) for x in df.values]
c = con.cursor()

Create a table without specifying data types

In [75]:
#Create a table without specifying data types
con.execute("CREATE TABLE sunsp(year, sunactivity)")

Insert records from a list of tuples – insert all the rows into the table and show the row count

In [76]:
#Insert records from a list of tuples – insert all the rows into the table and show the row count
con.executemany("INSERT INTO sunsp(year, sunactivity) VALUES (?, ?)", rows)


In [77]:
c.execute("SELECT COUNT(*) FROM sunsp")
print(c.fetchone())

(309,)


Delete the records where the sunactivity is greater than 20

In [93]:
#Delete the records where the sunactivity is greater than 20
print("Deleted", con.execute("DELETE FROM sunsp where sunactivity >20").rowcount, "rows")

Deleted 217 rows


Write a query to select all data where the year is less than 1732

In [80]:
#Write a query to select all data where the year is less than 1732
print(read_sql("SELECT * FROM sunsp WHERE year<1732", con))

      year  sunactivity
0   1700.0          5.0
1   1701.0         11.0
2   1702.0         16.0
3   1703.0         23.0
4   1704.0         36.0
5   1705.0         58.0
6   1706.0         29.0
7   1707.0         20.0
8   1708.0         10.0
9   1709.0          8.0
10  1710.0          3.0
11  1711.0          0.0
12  1712.0          0.0
13  1713.0          2.0
14  1714.0         11.0
15  1715.0         27.0
16  1716.0         47.0
17  1717.0         63.0
18  1718.0         60.0
19  1719.0         39.0
20  1720.0         28.0
21  1721.0         26.0
22  1722.0         22.0
23  1723.0         11.0
24  1724.0         21.0
25  1725.0         40.0
26  1726.0         78.0
27  1727.0        122.0
28  1728.0        103.0
29  1729.0         73.0
30  1730.0         47.0
31  1731.0         35.0


In [82]:
#Close your connection
c.close()

##### 3. Install sqlalchemy

In [84]:
#Install sqlalchemy
print("Run the 'pip3 install sqlalchemy' in terminal")

Run the 'pip3 install sqlalchemy' in terminal


Define the superclass

In [85]:
#Define the superclass
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

Create the DBSession

In [86]:
from sqlalchemy.orm import sessionmaker
from sqlalchemy import Column, ForeignKey, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship
from sqlalchemy import create_engine
from sqlalchemy import UniqueConstraint

In [95]:
Base = declarative_base()
class Station(Base):
    __tablename = 'station'
    id = Column(Integer, primary_key=True)
    name = Column(String(14), nullable = False, unique = True)

    def __repr__(self):
        return "Id=%d name=%d" %(self.id, self.name)

class Sensor(Base):
    __tablename__ = 'sensor'
    id = Column(Integer, primary_key = True)
    last = Column(Integer)
    multiplier = Column(Float)
    station_id = Column(Integer, ForeignKey('station.id'))
    station = relationship(Station)

    def __repr__(self):
        return "Id={:d} last={:d} multiplier={:.1f} station_id={:d}".format(
            self.id, self.last, self.multiplier, self.station_id)
if __name__ == "__main__":
    print("script use by code further")


NameError: name 'engine' is not defined

Create 2 stations and commit the session

In [95]:
#Create 2 stations and commit the session
premier = Station(name = 'premier')
session.add(premier)
session.add(Station(name= 'Utrecht'))
print("Station ", premier)

o	Insert a sensor record

In [ ]:
#Insert a sensor record
temp_sensor = Sensor(last = 20, multiplier=.1, station = premier)
session.add(temp_sensor)
session.commit()
print("Session", temp_sensor)

##### 4. Create an engine from a URI

In [90]:
#Create an engine from a URI
engine = create_engine('sqlite:///demo.db')


o	Create a DBSession object

In [91]:
from sqlalchemy.orm.sync import populate

#Create a DBSession object

Base.metadata.create_all(engine)
populate(engine)
Base.metadata.bind = engine
DBSession = sessionmaker()
DBSession.bind = engine
session = DBSession()



o	Select the first row in the station table

In [ ]:
#Select the first row in the station table

station = session.query(Station).first()

o	Select all the stations

In [ ]:
#Select all the stations
print("Query 1", session.query(Station).all())

o	Select all the sensors

In [ ]:
#Select all the sensors
print("Query 2", session.query(Sensor).all())

o	Select the first sensor, which belongs to the first station

In [ ]:
#Select the first sensor, which belongs to the first station
print("Query 3", session.query(Sensor).filter(Sensor.station == station).one())

o	Query the station table with pandas read_sql()

In [95]:
#Query the station table with pandas read_sql()
print(read_sql("SELECT * FROM station", engine.raw_connection()))